# 1.Scraping the Data

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 发送HTTP请求获取网页内容
url = 'https://www.studierendenwerk-muenchen-oberbayern.de/en/accommodation/private-accommodation-service/rooms-for-rent/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# 查找网页中的表格
table = soup.find('table')

# 提取表格中的数据
headers = [header.text for header in table.find_all('th')]
rows = []
for row in table.find_all('tr'):
    cells = row.find_all('td')
    if cells:
        rows.append([cell.text for cell in cells])

# 将数据保存为DataFrame
df = pd.DataFrame(rows, columns=headers)

# 保存为CSV文件
df.to_csv('table_data.csv', index=False)


In [20]:
# pip install gspread oauth2client


In [21]:
# pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


## use the data (address) ->

# 2. Calculate Commuting Time

In [24]:
from dotenv import load_dotenv
import os
# 加载环境变量
load_dotenv()

# 从环境变量获取Google Maps API密钥
API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')

In [13]:
import requests
import pandas as pd


# 指定地点的经纬度
destination = '48.15004, 11.56701'  # TUM school of management

# 从Google表格或CSV文件中加载位置数据
data = pd.read_csv('table_data - table_data.csv')  # 假设CSV文件中有'latitude'和'longitude'列

# 创建一个列表来存储结果
results = []

# 遍历每个地点，计算到指定地点的公共交通时间
for index, row in data.iterrows():
    origin = f"{row['Latitude']},{row['Longitude']}"
    url = f"https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&mode=transit&key={API_KEY}"
    
    response = requests.get(url)
    directions = response.json()
    
    if directions['status'] == 'OK':
        duration = directions['routes'][0]['legs'][0]['duration']['text']
        results.append({'latitude': row['Latitude'], 'longitude': row['Longitude'], 'duration': duration})
    else:
        results.append({'latitude': row['Latitude'], 'longitude': row['Longitude'], 'duration': 'N/A'})


# 将结果保存到CSV文件
results_df = pd.DataFrame(results)
results_df.to_csv('transit_times.csv', index=False)

print('公共交通时间计算完成并已保存到transit_times.csv')


公共交通时间计算完成并已保存到transit_times.csv


# 3. Visualize Commuting Time

In [19]:
# pip install folium

In [17]:
import pandas as pd
import folium

# 加载CSV数据
data = pd.read_csv('transit_times.csv').dropna()

# 创建一个基础地图
m = folium.Map(location=[48.15004, 11.56701], zoom_start=10)

# 定义颜色映射函数
def get_color(duration):
    if duration == 'N/A' or not isinstance(duration, str):
        return 'gray'
    
    # 将时间转换为总分钟数
    time_parts = duration.split()
    if 'hour' in time_parts or 'hours' in time_parts:
        hours = int(time_parts[0])
        minutes = int(time_parts[2]) if len(time_parts) > 2 else 0
        time = hours * 60 + minutes
    else:
        time = int(time_parts[0])

    if time < 30:
        return 'green'
    elif time < 60:
        return 'orange'
    else:
        return 'red'

# 添加标记到地图
for index, row in data.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color=get_color(row['duration']),
        fill=True,
        fill_color=get_color(row['duration']),
        fill_opacity=0.6,
        popup=f"Duration: {row['duration']}"
    ).add_to(m)

# 保存地图为HTML文件
m.save('transit_times_map.html')

print("地图已生成并保存为transit_times_map.html")


地图已生成并保存为transit_times_map.html
